We will perform dNdS using probabilities for single base substitutions (single nucleotide context)

Do we have patients with 2 mutations in a single gene?

General steps:
1. Get the probabilities for all 6 types of single base substitutions
   - C>A
   - C>G
   - C>T
   - T>A
   - T>C
   - T>G
2. 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# change working directory to project-2
if os.getcwd().split('/')[-1] != 'project-2':
    os.chdir('../../../')

In [12]:
# set paths
METADATA = 'data/raw/TCGA.BRCA.metadata.txt'
MUTATIONS = 'data/processed/TCGA.BRCA.mutations.qc1.txt'
CDS_LENGTHS = 'data/processed/gencode.v23lift37.pc_transcripts.transcripts_in_TCGA_MAF.cds_lengths.tsv'
dnds_opportunities = 'data/processed/dnds_opportunities.tsv'

df_opportunities = pd.read_csv(dnds_opportunities, sep='\t')
df_mut = pd.read_csv(MUTATIONS, sep='\t')
df_meta = pd.read_csv(METADATA, sep='\t')
df_cds = pd.read_csv(CDS_LENGTHS, sep='\t')

In [16]:
file = "data/processed/gencode.v23lift37.pc_transcripts.transcripts_in_TCGA_MAF.fa"
df = pd.read_csv(file, sep='\t')

In [6]:
df_cds.head()

,Hugo_Symbol,Transcript_ID,CDS_length,CDS sequence
0,A1BG,ENST00000263100,1488,ATGTCCATGCTCGTGGTCTTTCTCTTGCTGTGGGGTGTCACCTGGG...
1,A1CF,ENST00000373995,1785,ATGGAAGCAGTGTGTCTGGGCACATGCCCAGAGCCAGAAGCGAGCA...
2,A2M,ENST00000318602,4425,ATGGGGAAGAACAAACTCCTTCATCCAAGTCTGGTTCTTCTCCTCT...
3,A2ML1,ENST00000299698,4365,ATGTGGGCTCAGCTCCTTCTAGGAATGTTGGCCCTATCACCAGCCA...
4,A4GALT,ENST00000401850,1062,ATGTCCAAGCCCCCCGACCTCCTGCTGCGGCTGCTCCGGGGCGCCC...


In [ ]:
import re
from collections import defaultdict
import pandas as pd

# Mutation count table: SBS_count[ref][alt]
SBS_count = defaultdict(lambda: defaultdict(int))

def normalize_sbs(ref, alt):
    # Normalize purine ref bases to pyrimidine (G→C, A→T; and likewise alt)
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    if ref in ['G', 'A']:
        ref = complement[ref]
        alt = complement[alt]
    return ref, alt

for hgvsc in df_mut['HGVSc']:
    m = re.search(r'c\.\d+([ACGT])>([ACGT])', str(hgvsc))
    if not m:
        continue
    ref, alt = m.group(1), m.group(2)
    norm_ref, norm_alt = normalize_sbs(ref, alt)
    SBS_count[norm_ref][norm_alt] += 1

records = []
for ref in ['C', 'T']:
    for alt in ['A', 'G', 'T'] if ref == 'C' else ['A', 'C', 'G']:
        count = SBS_count[ref][alt]
        records.append({'ref': ref, 'alt': alt, 'mutation': f"{ref}>{alt}", 'count': count})

sbs_df = pd.DataFrame(records)

# Count total number of 'C' and 'T' bases across all CDSs
base_counts = defaultdict(int)

for seq in df_cds['CDS sequence']:
    seq = str(seq).upper()
    base_counts['C'] += seq.count('C')
    base_counts['T'] += seq.count('T')

# Multiply by 2 * number of individuals (diploid genomes)
num_individuals = len(df_meta)
multiplier = 2 * num_individuals

# Assign opportunities for each of the 6 SBS types
opportunities = {
    'C>A': base_counts['C'] * multiplier,
    'C>G': base_counts['C'] * multiplier,
    'C>T': base_counts['C'] * multiplier,
    'T>A': base_counts['T'] * multiplier,
    'T>C': base_counts['T'] * multiplier,
    'T>G': base_counts['T'] * multiplier
}

sbs_df['opportunities'] = sbs_df['mutation'].map(opportunities)
sbs_df['rate'] = sbs_df['count'] / sbs_df['opportunities']

In [ ]:
from collections import defaultdict
from itertools import product

# DNA codon table
codon_table = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',

    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',

    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',

    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
}


mutation_rate_map = sbs_df.set_index('mutation')['rate'].to_dict()

codon_opportunity = {}

for codon in codon_table:
    syn = 0.0
    nonsyn = 0.0
    ref_aa = codon_table[codon]

    for pos in range(3):
        ref_base = codon[pos]
        for alt_base in "ACGT":
            if alt_base == ref_base:
                continue
            mut_codon = codon[:pos] + alt_base + codon[pos+1:]
            if mut_codon not in codon_table:
                continue
            alt_aa = codon_table[mut_codon]

            norm_ref, norm_alt = normalize_sbs(ref_base, alt_base)
            key = f"{norm_ref}>{norm_alt}"
            rate = mutation_rate_map.get(key, 0.0)

            if ref_aa == alt_aa:
                syn += rate
            else:
                nonsyn += rate

    codon_opportunity[codon] = (syn, nonsyn)


In [ ]:
results = []

for idx, row in df_cds.iterrows():
    gene = row['gene']
    seq = row['CDS sequence'].upper()

    O_S = observed_synonymous.get(gene, 0)
    O_NS = observed_nonsynonymous.get(gene, 0)

    E_S, E_NS = 0.0, 0.0
    for i in range(0, len(seq) - 2, 3):
        codon = seq[i:i+3]
        if codon not in codon_opportunity:
            continue
        syn, nonsyn = codon_opportunity[codon]
        E_S += syn
        E_NS += nonsyn

    λ = O_S / E_S if E_S > 0 else None
    expected_NS_scaled = λ * E_NS if λ is not None else None
    dnds = O_NS / expected_NS_scaled if expected_NS_scaled else None

    results.append({
        'gene': gene,
        'O_S': O_S,
        'O_NS': O_NS,
        'E_S': E_S,
        'E_NS': E_NS,
        'λ': λ,
        'dN/dS': dnds
    })

df_dnds = pd.DataFrame(results)
